In [ ]:
# https://github.com/udacity/intro-to-ml-tensorflow/tree/master

In [ ]:
# %pip install tensorflow[and-cuda]
# %pip install tensorflow_datasets
# %pip install keras
# %pip install matplotlib
# %pip install scikit-learn

In [ ]:
import tensorflow as tf
import warnings
warnings.filterwarnings('ignore')
import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR) 
import keras
import numpy as np
import matplotlib.pyplot as plt
#%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import tensorflow_datasets as tfds
tfds.disable_progress_bar()

In [ ]:
print(tf.reduce_sum(tf.random.normal([1000, 1000])))

In [ ]:
print(tf.config.list_physical_devices('GPU'))

In [ ]:
print('Using:')
print('\t\u2022 TensorFlow version:', tf.__version__)
print('\t\u2022 tf.keras version:', keras.__version__)
print('\t\u2022 Running on GPU' if tf.test.is_gpu_available() else '\t\u2022 GPU device not found. Running on CPU')

In [ ]:
# https://www.tensorflow.org/datasets/overview
iris_data, iris_info = tfds.load('iris', with_info=True, as_supervised=True)
print(iris_info)

In [ ]:
# Extract features from the iris training dataset
features = []
for feature, label in iris_data['train']:
    features.append(feature.numpy())
features = np.array(features)
features

In [ ]:
# Get the label description from iris_info
label_description = iris_info.features['label'].names
print(label_description)

In [ ]:
num_classes = iris_info.features['label'].num_classes
print('There are {:,} classes in our dataset'.format(num_classes))

num_training_examples = iris_info.splits['train'].num_examples
print('\nThere are {:,} rows in the training set'.format(num_training_examples))

In [ ]:
# creating pipelines 
# def normalize(image, label):
#     image = tf.cast(image, tf.float32)
#     image /= 255
#     return image, label

# batch_size = 64

# training_batches = training_set.cache().shuffle(num_training_examples//4).batch(batch_size).map(normalize).prefetch(1)

In [ ]:
# https://www.tensorflow.org/guide/data_performance
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset

In [ ]:
# model = tf.keras.Sequential([
#         tf.keras.layers.Flatten(input_shape = (28,28,1)),
#         tf.keras.layers.Dense(256, activation = 'sigmoid'),
#         tf.keras.layers.Dense(10, activation = 'softmax')
# ])

# model.summary()

In [ ]:
batch_size = 4

def preprocess(features, label):
    features = tf.cast(features, tf.float32)
    return features, label

train_ds = iris_data['train'] \
    .map(preprocess) \
    .cache() \
    .shuffle(num_training_examples) \
    .batch(batch_size) \
    .prefetch(tf.data.AUTOTUNE)

In [ ]:
model = keras.Sequential([
    keras.layers.Dense(16, activation='relu', input_shape=(4,)),
    keras.layers.Dense(12, activation='relu'),
    keras.layers.Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

In [ ]:
# https://keras.io/api/layers/initializers/
# https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/Dense

In [ ]:
weights_and_biases = model.get_weights()
len(weights_and_biases)

In [ ]:
model.layers

In [ ]:
weights = model.get_layer(index=0).get_weights()[0]
weights

In [ ]:
biases = model.get_layer(index=0).get_weights()[1]
biases

In [ ]:
from keras import Model

# Subclassing 
class Network_1(Model):
    def __init__(self, num_classes = 2):
        super().__init__()
        self.num_classes = num_classes
    
        # Define layers 
        self.input_layer = keras.layers.Flatten()
        self.hidden_layer = keras.layers.Dense(256, activation = 'relu')
        self.output_layer = keras.layers.Dense(self.num_classes, activation = 'softmax')
    
    # Define forward Pass   
    def call(self, input_tensor):
        x = self.input_layer(input_tensor)
        x = self.hidden_layer(x)
        x = self.output_layer(x)
    
        return x

In [ ]:
# Create a model object
subclassed_model = Network_1(10)

# Build the model, i.e. initialize the model's weights and biases
subclassed_model.build((None, 28, 28, 1))

subclassed_model.summary()


In [ ]:
# adding layers
model = keras.Sequential()

model.add(keras.layers.Flatten(input_shape = (28,28,1)))
model.add(keras.layers.Dense(32, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))
          
model.summary()

In [ ]:
layer_neurons = [512, 256, 128, 56, 28, 14]

model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape = (28,28,1)))

for neurons in layer_neurons:
    model.add(keras.layers.Dense(neurons, activation='relu'))
            
model.add(keras.layers.Dense(10, activation='softmax'))
          
model.summary() 

In [ ]:
# clearing the graph
keras.backend.clear_session()

layer_neurons = [512, 256, 128, 56, 28, 14]

model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape = (28,28,1)))

for neurons in layer_neurons:
    model.add(keras.layers.Dense(neurons, activation='relu'))
            
model.add(keras.layers.Dense(10, activation='softmax'))
          
model.summary() 